Generate all possible Ugi products for a set of reactants and return a csv file with their smiles, chemical formula and protonated monoisotopic mass. It also generates a .png image with all the possible structures and calculates each product MA. (rdkit, pandas, massmol and assemblycalculatormodules are required)

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions
from rdkit.Chem.SimpleEnum import Enumerator
from rdkit.Chem import rdMolDescriptors
import pandas as pd
import numpy as np
from molmass import Formula
import assemblycalculator as ac

Now define functions to take 

In [3]:
def get_reactants(file):
    
    '''Takes a txt file with the SMILES of the reactants and creates a list of lists with them'''
    
    with open(file,'r') as f:
        lines = f.read()
        mol = lines.split('\n')
        
        reactants=[]
        
        for line in mol:
            mols = line.split('.')
            reactants.append(mols)
        
    return reactants


def smile_to_mol(smiles):
    '''Transform a list of SMILES into a list of Mols'''

    mols = [Chem.MolFromSmiles(smile) for smile in smiles]

    return mols

 
def product_generator(reactants):
    
    '''Generates all possible products for a set of reactants in a given reaction (Ugi in this case, defined by the
    SMARTS) and returns a list containing their SMILES.'''
    
    Ugi = AllChem.ReactionFromSmarts('[#6:1][NH2:2].[CX3H:3]=[O:7].[CX3:4](=[O:8])[OH:9].[CX4:99][N+:5]#[C-:6]>>[C:4](=[O:8])[N:2]([*:1])[*:3][C+0:6](=[O:9])[NX2H+0:5][CX4:99].[OH2:7]')
    
    products = Enumerator.EnumerateReaction(Ugi, reactants , uniqueProductsOnly=True)
    
    product_smiles = [Chem.MolToSmiles(product[0]) for product in products]
    
    return product_smiles


Run code

In [4]:
reactants = get_reactants('test_smiles.txt')

reactants = [smile_to_mol(fgroup) for fgroup in reactants]

product_smiles = product_generator(reactants)

Calculate the molecular formula and the protonated monoisotopic peak for all the products

In [5]:
def get_molecular_formula(smiles):
    '''This function takes a list of SMILES and returns another list with their
    corresponding molecular formulas'''

    formulas = [rdMolDescriptors.CalcMolFormula(Chem.MolFromSmiles(smile)) for smile in smiles]

    return formulas

def estimate_monoisotopic_peak(molformulas):
    '''This function takes a list of molecular formulas and returns a list 
    with their estimated monoisotopic peaks (adding one H+ due to positive ion
    mode'''

    peaks = [(Formula(molecule).isotope.mass + 1.007825) for molecule in molformulas]

    return peaks


formulas = get_molecular_formula(product_smiles)

peaks = estimate_monoisotopic_peak(formulas)


Create a pd.DataFrame and include the data

In [7]:
data = pd.DataFrame({'Product SMILES': product_smiles, 'empirical formula': formulas,
                         'monoisotopic peak': peaks})


data.head()

,Product SMILES,empirical formula,monoisotopic peak
0,C=CCN(C(=O)c1ccccc1)C(CCCC)C(=O)NC1CCCCC1,C22H32N2O2,357.254203
1,C=CCN(C(=O)c1ccccc1)C(CCCC)C(=O)NC(C)(C)C,C20H30N2O2,331.238553
2,C=CCN(C(=O)CCCCC)C(CCCC)C(=O)NC1CCCCC1,C21H38N2O2,351.301153
3,C=CCN(C(=O)CCCCC)C(CCCC)C(=O)NC(C)(C)C,C19H36N2O2,325.285503
4,C=CCN(C(=O)C1CCCCC1)C(CCCC)C(=O)NC1CCCCC1,C22H38N2O2,363.301153


Draw the products and save them as a .png image

In [8]:
products_mol = [Chem.MolFromSmiles(product) for product in product_smiles]
    
images = Draw.MolsToGridImage(products_mol, molsPerRow=8, returnPNG=False, legends = formulas)

images.save('products.png')

Finally, the MA of each product is calculated using the Monte Carlo method of the assemblycalculator module. The results are included in the DataFrame and saved as a csv file

In [9]:
mas = []

for smile in product_smiles:
    
    ma = ac.calculate_ma(smile, method= "monte-carlo", timeout = 3600, num_frags_hist = 1e6, path_samples = 1e5)
    mas.append(ma)

data['MA'] = mas

data.to_csv('products_MA.csv', encoding = 'utf-8')


In [10]:
data.head()

,Product SMILES,empirical formula,monoisotopic peak,MA
0,C=CCN(C(=O)c1ccccc1)C(CCCC)C(=O)NC1CCCCC1,C22H32N2O2,357.254203,15
1,C=CCN(C(=O)c1ccccc1)C(CCCC)C(=O)NC(C)(C)C,C20H30N2O2,331.238553,16
2,C=CCN(C(=O)CCCCC)C(CCCC)C(=O)NC1CCCCC1,C21H38N2O2,351.301153,11
3,C=CCN(C(=O)CCCCC)C(CCCC)C(=O)NC(C)(C)C,C19H36N2O2,325.285503,14
4,C=CCN(C(=O)C1CCCCC1)C(CCCC)C(=O)NC1CCCCC1,C22H38N2O2,363.301153,12
